# 02. Preprocessing of text data

In [5]:
import pandas as pd
import numpy as np
import re
import os
import nltk
from nltk.corpus import stopwords
from natasha import Segmenter, NewsEmbedding, NewsMorphTagger, MorphVocab, Doc

In [6]:
try:
    stopwords.words('russian')
except LookupError:
    nltk.download('stopwords')

print("Все библиотеки загружены!")

Все библиотеки загружены!


## 1. data loading

In [7]:
data_path = os.path.join('..', 'sample_data', 'sample_inquiries.csv')

try:
    df = pd.read_csv(data_path)
    print(f"Датасет успешно загружен из {data_path}")
    print(f"Размер: {df.shape}")
except FileNotFoundError:
    print(f"Файл не найден по пути: {data_path}")
    df = pd.DataFrame(columns=['text', 'category'])

df.head()

Датасет успешно загружен из ..\sample_data\sample_inquiries.csv
Размер: (98, 2)


,text,category
0,"Здравствуйте, как я могу получить адресную спр...",Запрос на госуслугу
1,Почему портал электронного правительства посто...,Техническая проблема
2,Выражаю благодарность сотруднику ЦОНа №3 город...,Предложение/Отзыв
3,Не могу записаться на прием в поликлинику чере...,Техническая проблема
4,Какие документы необходимы для регистрации рож...,Запрос на госуслугу


## 2. Create function for preprocessing text

In [8]:
segmenter = Segmenter()
morph_tagger = NewsMorphTagger(NewsEmbedding())
morph_vocab = MorphVocab()

In [9]:
russian_stopwords = set(stopwords.words("russian"))

def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)
    text = text.lower()
    
    text = re.sub(r'[^а-яё\s]', ' ', text)

    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    lemmas = []
    for token in doc.tokens:
        if token.text not in russian_stopwords:
            token.lemmatize(morph_vocab)
            lemmas.append(token.lemma)

    return lemmas

## 3. Processing all dataset

In [10]:
print("Предобработка текстов...")
df['tokens'] = df['text'].apply(preprocess_text)
print("Готово.")

df['processed_text'] = df['tokens'].apply(lambda tokens: ' '.join(tokens))

Предобработка текстов...
Готово.


In [11]:
df[['text', 'processed_text', 'category']].head(10)

,text,processed_text,category
0,"Здравствуйте, как я могу получить адресную спр...",здравствовать мочь получить адресный справка о...,Запрос на госуслугу
1,Почему портал электронного правительства посто...,почему портал электронный правительство постоя...,Техническая проблема
2,Выражаю благодарность сотруднику ЦОНа №3 город...,выражать благодарность сотрудник цон город аст...,Предложение/Отзыв
3,Не могу записаться на прием в поликлинику чере...,мочь записаться прием поликлиника мобильный пр...,Техническая проблема
4,Какие документы необходимы для регистрации рож...,какой документ необходимый регистрация рождени...,Запрос на госуслугу
5,Ужасное обслуживание! Оператор горячей линии б...,ужасный обслуживание оператор горячий линия оч...,Жалоба
6,"Предлагаю улучшить навигацию на вашем сайте, о...",предлагать улучшить навигация ваш сайт очень с...,Предложение/Отзыв
7,Мое обращение по поводу ремонта дороги зарегис...,мой обращение повод ремонт дорога зарегистриро...,Жалоба
8,"Как получить справку о несудимости, если я нах...",получить справка несудимость находиться границ...,Запрос на госуслугу
9,Неверно указана моя фамилия в базе данных. Как...,неверно указать фамилия база данные исправить ...,Техническая проблема


In [12]:
processed_data_dir = os.path.join('..', 'processed_data')
output_path = os.path.join(processed_data_dir, 'inquiries_processed.csv')

if not os.path.exists(processed_data_dir):
    os.makedirs(processed_data_dir)

df.to_csv(output_path, index=False, encoding='utf-8')
print(f"Файл сохранён: {output_path}")

Файл сохранён: ..\processed_data\inquiries_processed.csv


In [4]:
import pandas as pd
import os

processed_data_path = os.path.join('..', 'processed_data', 'inquiries_processed.csv')

df = pd.read_csv(processed_data_path)

sample = df[['text', 'processed_text']].dropna().sample(1).iloc[0]

print("До предобработки:\n")
print(sample['text'])

print("\nПосле предобработки:\n")
print(sample['processed_text'])


До предобработки:

Какие сроки рассмотрения заявления на получение пособия по уходу за ребенком до года?

После предобработки:

какой срок рассмотрение заявление получение пособие уход ребенок год
